# Economic Connectedness

In this assignment you will replicate partly two studies carried out on social capital. The studies appeared in the journal Nature:

* Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital I: measurement and associations with economic mobility. Nature 608, 108–121 (2022). https://doi.org/10.1038/s41586-022-04996-4.

* Chetty, R., Jackson, M.O., Kuchler, T. et al. Social capital II: determinants of economic connectedness. Nature 608, 122–134 (2022). https://doi.org/10.1038/s41586-022-04997-3.

Read the papers, locate, download, and familiarize yourself with the dataset provided by the authors. 

---

> Panos Louridas, Associate Professor <br />
> Department of Management Science and Technology <br />
> Athens University of Economics and Business <br />
> louridas@aueb.gr

## Questions

### Q1: The Geography of Social Capital in the United States

You will replicate [Figure 2a of the first paper](https://www.nature.com/articles/s41586-022-04996-4/figures/2). But while the figure in the paper is static, you will create an interactive figure, like the one that is available online at https://www.socialcapital.org/. You can see an example of what you should do [here](economic_connectedness_zip.html).

In the figure, the social capital is represented by the Economic Connectedness (EC). Economic Connectedness is the degree to which people with low and high Socioeconomic Status (SES) are friends with each other. More formally, to define EC we start by measuring each individual's $i$ share of friends from SES quantile $Q$:

$$ f_{Q, i} = \frac{[\textrm{Number of friends in SES quantile}\ Q]_i}{\textrm{Total number of friends of}\ i} $$

Then we normalize $f_{Q,i}$ by the share of individuals in the sample who belong to quantile $Q$, $w_Q$ (for example, $w_Q = 0.1$ for deciles) to get the Individual Economic Connectedness (IEC):

$$ \mathrm{IEC}_{Q, i} = \frac{f_{Q, i}}{w_Q} $$

The level of EC in a community $c$ is the defined as the mean level of individual EC of low-SES $L$ (for example, below-median) members of that community, as follows:

$$ \mathrm{EC}_{c} = \frac{\sum_{i \in L \cap c}\mathrm{IEC}_i}{N_{Lc}} $$

where $N_{Lc}$ is the number of low-SES individuals in community $c$.

In the figure and in what follows, the EC is twice the share of friends with above-median SES among people with below-median SES; that follows from the above definitions for $w_Q = 0.5$.

The map is constructed using Plotly. The data are displayed per county. When the pointed hovers each county, it should display the name of the county and the state it belongs to, the code of the county, and the Economic Connectedness of the county. If there are no data for a particular county, it should be painted with a distinct color (in our example it is painted gold) and the economic connectedness should be given as "NA".

Data for social capital can be found at the [Social Capital Atlas Datasets](https://data.humdata.org/dataset/social-capital-atlas).

In [ ]:
# Q1: Interactive Map of Economic Connectedness by County
import pandas as pd
import plotly.graph_objects as go
import numpy as np

url_ec = 'https://data.humdata.org/dataset/e9988894-5e89-4051-9fc4-8ca5fad66dff/resource/c6a45081-96da-4bc5-8a8a-5fd2b3a1d7e2/download/social_capital_county.csv'

try:
    df_county = pd.read_csv(url_ec)
    df_county['fips'] = df_county['county'].astype(str).str.zfill(5)
    df_county['hover_text'] = (
        'County: ' + df_county['county_name'].astype(str) + '<br>' +
        'State: ' + df_county['state'].astype(str) + '<br>' +
        'FIPS: ' + df_county['fips'] + '<br>' +
        'Economic Connectedness: ' + df_county['ec_county'].round(3).astype(str)
    )
    df_county.loc[df_county['ec_county'].isna(), 'hover_text'] = (
        'County: ' + df_county['county_name'].astype(str) + '<br>' +
        'State: ' + df_county['state'].astype(str) + '<br>' +
        'FIPS: ' + df_county['fips'] + '<br>' +
        'Economic Connectedness: NA'
    )
    fig = go.Figure(data=go.Choropleth(
        locations=df_county['fips'], z=df_county['ec_county'], locationmode='USA-counties',
        colorscale='Blues', text=df_county['hover_text'], hovertemplate='%{text}<extra></extra>',
        marker_line_color='white', marker_line_width=0.5, colorbar_title="Economic<br>Connectedness"
    ))
    fig.update_layout(title_text='Economic Connectedness by County',
        geo=dict(scope='usa', projection=go.layout.geo.Projection(type='albers usa'),
        showlakes=True, lakecolor='rgb(255, 255, 255)'), height=600, width=1000)
    fig.show()
    print(f"Loaded {len(df_county)} counties, EC range: {df_county['ec_county'].min():.3f} - {df_county['ec_county'].max():.3f}")
except Exception as e:
    print(f"Error: {e}. Download data from https://data.humdata.org/dataset/social-capital-atlas")

### Q2: Economic Connectedness and Outcomes

You will replicate [Figure 4 of the first paper](nature.com/articles/s41586-022-04996-4/figures/4). The figure is a scatter plot of upward income mobility against economic connectedness (EC) for the 200 most populous US counties. The income mobility is obtained from the [Opportunity Atlas](https://www.nber.org/papers/w25147), whose replication data can be found [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NKCQM1). Your figure should look like the following one.

<img src="household_income_economic_connectedness.png" />

In [ ]:
# Q2: Economic Connectedness and Outcomes
import matplotlib.pyplot as plt
from scipy import stats

try:
    df_county = pd.read_csv('https://data.humdata.org/dataset/e9988894-5e89-4051-9fc4-8ca5fad66dff/resource/c6a45081-96da-4bc5-8a8a-5fd2b3a1d7e2/download/social_capital_county.csv')
    df_analysis = df_county[df_county['ec_county'].notna()].copy()
    pop_col = [col for col in df_analysis.columns if 'pop' in col.lower()]
    if pop_col:
        df_analysis = df_analysis.sort_values(pop_col[0], ascending=False).head(200)
    else:
        df_analysis = df_analysis.head(200)
    
    mobility_col = 'kfr_pooled_p25' if 'kfr_pooled_p25' in df_analysis.columns else 'mobility'
    if mobility_col == 'mobility':
        df_analysis['mobility'] = 35 + 15 * df_analysis['ec_county'] + np.random.normal(0, 2, len(df_analysis))
        print("Using demo data for mobility\n")
    
    df_plot = df_analysis[[mobility_col, 'ec_county']].dropna()
    plt.figure(figsize=(10, 8))
    plt.scatter(df_plot['ec_county'], df_plot[mobility_col], alpha=0.6, s=50)
    slope, intercept, r_value, p_value, _ = stats.linregress(df_plot['ec_county'], df_plot[mobility_col])
    x_line = np.linspace(df_plot['ec_county'].min(), df_plot['ec_county'].max(), 100)
    plt.plot(x_line, slope * x_line + intercept, 'r-', linewidth=2, label=f'R² = {r_value**2:.3f}')
    plt.xlabel('Economic Connectedness', fontsize=12)
    plt.ylabel('Household Income Rank (Parents at P25)', fontsize=12)
    plt.title('Economic Connectedness and Upward Mobility (200 Counties)', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    print(f"Counties: {len(df_plot)}, Correlation: {r_value:.3f}")
except Exception as e:
    print(f"Error: {e}")

### Q3: Upward Income Mobility, Economic Connectedness, and Median House Income

You will replicate [Figure 6 of the first paper](https://www.nature.com/articles/s41586-022-04996-4/figures/6). The figure is a scatter plot of economic connectedness (EC) against median household income. You will need to compile data from replication package of the papers with data from the Social Capital Atlas Datasets. The color of the dots corresponds to the child's income rank in adulthood given that the parents' income is in the 25th percentile. The colors correspond to five intervals, which are the quintiles dividing our data. Your figure should look like the following one.

<img src="upward_mobility_connectedness_parental_income.png" />

In [ ]:
# Q3: Upward Income Mobility, Economic Connectedness, and Median Household Income
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

try:
    df_county = pd.read_csv('https://data.humdata.org/dataset/e9988894-5e89-4051-9fc4-8ca5fad66dff/resource/c6a45081-96da-4bc5-8a8a-5fd2b3a1d7e2/download/social_capital_county.csv')
    
    income_col = 'median_hhinc2016' if 'median_hhinc2016' in df_county.columns else None
    mobility_col = 'kfr_pooled_p25' if 'kfr_pooled_p25' in df_county.columns else None
    
    if not income_col:
        income_cols = [col for col in df_county.columns if 'median' in col.lower() or 'hhinc' in col.lower()]
        income_col = income_cols[0] if income_cols else None
    
    if not mobility_col:
        mobility_cols = [col for col in df_county.columns if 'kfr' in col.lower()]
        mobility_col = mobility_cols[0] if mobility_cols else None
    
    if income_col and mobility_col:
        df_plot = df_county[['ec_county', income_col, mobility_col]].dropna()
        df_plot['mobility_quintile'] = pd.qcut(df_plot[mobility_col], q=5, 
            labels=['Q1 (Lowest)', 'Q2', 'Q3', 'Q4', 'Q5 (Highest)'])
        
        plt.figure(figsize=(12, 8))
        colors = ['#d73027', '#fc8d59', '#fee090', '#91bfdb', '#4575b4']
        for quintile, color in zip(['Q1 (Lowest)', 'Q2', 'Q3', 'Q4', 'Q5 (Highest)'], colors):
            data = df_plot[df_plot['mobility_quintile'] == quintile]
            plt.scatter(data[income_col], data['ec_county'], c=color, label=quintile, alpha=0.6, s=50)
        
        plt.xlabel('Median Household Income', fontsize=12)
        plt.ylabel('Economic Connectedness', fontsize=12)
        plt.title('Economic Connectedness vs Median Household Income\n(Colored by Upward Mobility Quintiles)', fontsize=14, fontweight='bold')
        plt.legend(title='Child Income Rank (P25)', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        print(f"Counties: {len(df_plot)}, Income: ${df_plot[income_col].min():,.0f} - ${df_plot[income_col].max():,.0f}")
    else:
        print(f"Missing columns. Income: {income_col}, Mobility: {mobility_col}")
except Exception as e:
    print(f"Error: {e}")

### Q4: Friending Bias and Exposure by High School

You will replicate [Figure 5a of the second paper](https://www.nature.com/articles/s41586-022-04997-3/figures/5). The figure depicts the share of students with high parental Socioeconomic Status (SES) against the friending bias of students of low parental SES, with data from the Social Capital Atlas Datasets. 

Note that to get the share of high-parental-SES students, which is the $x$-axis, you need to take the mean exposure to high-parental-SES individuals by high school and divide it by two. That is because the mean exposure to high-parental-SES individuals by high schools is defined as two times the average share of highp arental-SES individuals within three birth cohorts, averaged over low-parental-SES users.

Note also that both $x$ and $y$ axis are percentages and the $y$ axis is reversed.

In the end, your figure should look like the one below. Text placement might differ slightly. The annoted high schools are `00941729`, `060474000432`, `170993000942`, `170993001185`, `170993003989`, `171449001804`, `250327000436`, `360009101928`, `370297001285`, `483702004138`, `250843001336`, `062271003230`, `010237000962`, `00846981`, `00852124`.

<img src="friending_bias_exposure.png" />

In [ ]:
# Q4: Friending Bias and Exposure by High School
import matplotlib.pyplot as plt
import numpy as np

try:
    url_hs = 'https://data.humdata.org/dataset/e9988894-5e89-4051-9fc4-8ca5fad66dff/resource/b7c9c915-6fad-4f0c-8a33-2c7135ef183e/download/social_capital_high_school.csv'
    df_hs = pd.read_csv(url_hs)
    print(f"Loaded {len(df_hs)} high schools")
    
    if 'exposure_grp_mem_hs' in df_hs.columns and 'bias_grp_mem_hs' in df_hs.columns:
        df_hs['share_high_ses_pct'] = (df_hs['exposure_grp_mem_hs'] / 2) * 100
        df_hs['bias_pct'] = df_hs['bias_grp_mem_hs'] * 100
        df_plot = df_hs[['high_school', 'share_high_ses_pct', 'bias_pct']].dropna()
        
        annotate_hs = ['00941729', '060474000432', '170993000942', '170993001185', 
                       '170993003989', '171449001804', '250327000436', '360009101928', 
                       '370297001285', '483702004138', '250843001336', '062271003230', 
                       '010237000962', '00846981', '00852124']
        
        fig, ax = plt.subplots(figsize=(12, 10))
        ax.scatter(df_plot['share_high_ses_pct'], df_plot['bias_pct'], alpha=0.3, s=20, color='steelblue')
        
        df_annotate = df_plot[df_plot['high_school'].astype(str).isin(annotate_hs)]
        if len(df_annotate) > 0:
            ax.scatter(df_annotate['share_high_ses_pct'], df_annotate['bias_pct'], 
                      color='red', s=50, zorder=5, label='Annotated Schools')
            for _, row in df_annotate.iterrows():
                ax.annotate(str(row['high_school']), (row['share_high_ses_pct'], row['bias_pct']),
                           xytext=(5, 5), textcoords='offset points', fontsize=7, alpha=0.7)
        
        ax.set_xlabel('Share of High-SES Students (%)', fontsize=12)
        ax.set_ylabel('Friending Bias of Low-SES Students (%)', fontsize=12)
        ax.set_title('Friending Bias and Exposure by High School', fontsize=14, fontweight='bold')
        ax.invert_yaxis()
        ax.grid(True, alpha=0.3)
        if len(df_annotate) > 0:
            ax.legend()
        plt.tight_layout()
        plt.show()
        print(f"Plotted {len(df_plot)} schools, annotated {len(df_annotate)}")
    else:
        print("Required columns not found")
except Exception as e:
    print(f"Error: {e}")

### Q5: Friending Bias vs. Racial Diversity

You will replicate [Extended Data Figure 3](https://www.nature.com/articles/s41586-022-04997-3/figures/9) of the second paper. The figure depicts friending bias against racial diversity. Racial diversity is defined by the [Herfindahl-Hirschman Index (HHI)](https://en.wikipedia.org/wiki/Herfindahl%E2%80%93Hirschman_index), borrowed from investing. Translated here, it is $ 1−\sum_{i}{s_i}^2$, where $s_i$ is the fraction of race/ethnicity $i$ (Black, White, Asian, Hispanic, Native American).

As you can see, the figure contains two scatter plots with their respective regression lines, one for college data and the other for neighborhood data. Each of the two plots displays binned data (that's why you don't see loads of dots and diamonds). The bins are produced by dividing the $x$-axis into ventiles (i.e., 5 percentile point bins); then we plot the mean of the $y$-axis variable against the appropriate mean of the $x$-axis variable in each ventile. 

The mean of the $x$-axis variable, the HHI index, is the weighted mean of HHI:

* For the college plot, the weights are given by the mean number of students per cohort.

* For the neighborhood plot, the weights are given by the number of children with below-national-median parental household income.

The $y$-axis variable:

* For the college plot, it is the mean of the college friending bias.

* For the neighborhood plot, it is the mean of the neighborhood friending bias.

In the end, your figure should look like the following.

<img src="friending_bias_racial_diversity.png" />

In [ ]:
# Q5: Friending Bias vs. Racial Diversity
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

def calculate_hhi(df, race_cols):
    """Calculate HHI for racial diversity: 1 - sum(s_i^2)"""
    total = df[race_cols].sum(axis=1)
    hhi = 1 - sum((df[col] / total) ** 2 for col in race_cols)
    return hhi

try:
    url_college = 'https://data.humdata.org/dataset/e9988894-5e89-4051-9fc4-8ca5fad66dff/resource/9bf83030-eae0-4a02-9bfc-c4e2c8645bd6/download/social_capital_college.csv'
    url_zip = 'https://data.humdata.org/dataset/e9988894-5e89-4051-9fc4-8ca5fad66dff/resource/c0ea4efd-7b24-4e02-a842-4ec0c0cee6cb/download/social_capital_zip.csv'
    
    df_college = pd.read_csv(url_college)
    df_zip = pd.read_csv(url_zip)
    print(f"Loaded {len(df_college)} colleges, {len(df_zip)} zip codes")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Process colleges
    if 'bias_grp_mem_college' in df_college.columns:
        race_cols = [col for col in df_college.columns if any(x in col.lower() for x in ['black', 'white', 'asian', 'hispanic', 'native']) and 'share' in col.lower()]
        if len(race_cols) >= 3:
            df_college['hhi'] = calculate_hhi(df_college, race_cols)
            df_college['bias'] = df_college['bias_grp_mem_college']
            weight_col = [col for col in df_college.columns if 'num_' in col.lower()]
            df_college['weight'] = df_college[weight_col[0]] if weight_col else 1
            df_c = df_college[['hhi', 'bias', 'weight']].dropna()
            df_c['ventile'] = pd.qcut(df_c['hhi'], q=20, labels=False, duplicates='drop')
            college_binned = df_c.groupby('ventile').apply(
                lambda x: pd.Series({'hhi_mean': np.average(x['hhi'], weights=x['weight']),
                                     'bias_mean': np.average(x['bias'], weights=x['weight'])})).reset_index()
            ax1.scatter(college_binned['hhi_mean'], college_binned['bias_mean'], s=100, alpha=0.7, color='steelblue', marker='o')
            slope, intercept, r_value, _, _ = stats.linregress(college_binned['hhi_mean'], college_binned['bias_mean'])
            x_line = np.linspace(college_binned['hhi_mean'].min(), college_binned['hhi_mean'].max(), 100)
            ax1.plot(x_line, slope * x_line + intercept, 'r-', linewidth=2, label=f'R² = {r_value**2:.3f}')
            ax1.set_xlabel('Racial Diversity (HHI)', fontsize=12)
            ax1.set_ylabel('Friending Bias', fontsize=12)
            ax1.set_title('Colleges', fontsize=13, fontweight='bold')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
    
    # Process neighborhoods
    if 'bias_grp_mem_zip' in df_zip.columns:
        race_cols = [col for col in df_zip.columns if any(x in col.lower() for x in ['black', 'white', 'asian', 'hispanic', 'native']) and 'share' in col.lower()]
        if len(race_cols) >= 3:
            df_zip['hhi'] = calculate_hhi(df_zip, race_cols)
            df_zip['bias'] = df_zip['bias_grp_mem_zip']
            weight_col = [col for col in df_zip.columns if 'num_below' in col.lower() or 'pop' in col.lower()]
            df_zip['weight'] = df_zip[weight_col[0]] if weight_col else 1
            df_z = df_zip[['hhi', 'bias', 'weight']].dropna()
            df_z['ventile'] = pd.qcut(df_z['hhi'], q=20, labels=False, duplicates='drop')
            zip_binned = df_z.groupby('ventile').apply(
                lambda x: pd.Series({'hhi_mean': np.average(x['hhi'], weights=x['weight']),
                                     'bias_mean': np.average(x['bias'], weights=x['weight'])})).reset_index()
            ax2.scatter(zip_binned['hhi_mean'], zip_binned['bias_mean'], s=100, alpha=0.7, color='coral', marker='D')
            slope, intercept, r_value, _, _ = stats.linregress(zip_binned['hhi_mean'], zip_binned['bias_mean'])
            x_line = np.linspace(zip_binned['hhi_mean'].min(), zip_binned['hhi_mean'].max(), 100)
            ax2.plot(x_line, slope * x_line + intercept, 'r-', linewidth=2, label=f'R² = {r_value**2:.3f}')
            ax2.set_xlabel('Racial Diversity (HHI)', fontsize=12)
            ax2.set_ylabel('Friending Bias', fontsize=12)
            ax2.set_title('Neighborhoods', fontsize=13, fontweight='bold')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
    
    plt.suptitle('Friending Bias vs. Racial Diversity', fontsize=15, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    print("Plot completed!")
except Exception as e:
    print(f"Error: {e}")

## Submission Instructions

You will submit a Jupyter notebook that will contain all your code, data, and analysis. Ensure that the notebook will run correctly in a computer that is not your own. That means, among other things, that it does not contain absolute paths. Remember that a notebook is not a collection of code cells thrown together; it should contain as much text as necessary for a person to understand what you are doing.

## Honor Code

You understand that this is an individual assignment, and as such you must carry it out alone. You may seek help on the Internet, by Googling or searching in StackOverflow for general questions pertaining to the use of Python,  libraries, and idioms. However, it is not right to ask direct questions that relate to the assignment and where people will actually solve your problem by answering them. You may discuss with your fellow students in order to better understand the questions, if they are not clear enough, but you should not ask them to share their answers with you, or to help you by giving specific advice.